In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import  types
from pyspark.sql import functions as F
import pandas as pd

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
spark.version

In [33]:
df_raw = pd.read_csv('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/raw/FHVHV/fhvhv_tripdata_2021-06.csv.gz', nrows=400)

In [34]:
df_raw

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN
...,...,...,...,...,...,...,...
395,B02866,2021-06-01 00:30:38,2021-06-01 00:51:53,80,62,N,B02866
396,B02875,2021-06-01 00:06:23,2021-06-01 00:22:04,136,60,N,B02875
397,B02875,2021-06-01 00:25:31,2021-06-01 00:33:30,60,212,N,B02875
398,B02875,2021-06-01 00:49:57,2021-06-01 01:01:04,75,42,N,B02875


In [38]:
df_raw['Affiliated_base_number'] = df_raw['Affiliated_base_number'].fillna('')

In [43]:
spark.createDataFrame(df_raw).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [14]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number  ', types.StringType(), True)])

In [15]:
df_spark = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/raw/FHVHV/fhvhv_tripdata_2021-06.csv.gz')

In [17]:
df_spark \
    .repartition(12) \
    .write.parquet('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/pq/fhvhv', mode='overwrite')

In [44]:
!tree -h /home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/pq/fhvhv
!ls -lah /home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/pq/fhvhv

/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/pq/fhvhv
├── [   0]  _SUCCESS
├── [ 24M]  part-00000-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00001-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00002-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00003-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00004-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00005-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00006-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00007-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00008-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00009-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
├── [ 24M]  part-00010-c6ffeda5-de2c-4cb1-9899-d3fad16324ed-c000.snappy.parquet
└── [ 24M]  part-0001

In [27]:
df_park = spark.read.parquet('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/pq/fhvhv')

In [28]:
df_park.registerTempTable('spark_table')

/home/Дмитрий/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [50]:
df_result = spark.sql(
"""
select count(*)
from spark_table  
where pickup_datetime >= '2021-06-15' 
      and pickup_datetime < '2021-06-16'
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [75]:
spark.sql(
"""
select *,
(unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime))/3600 as hour
from spark_table  
order by hour desc
limit 10
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+------------------------+------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number  |              hour|
+--------------------+-------------------+-------------------+------------+------------+-------+------------------------+------------------+
|              B02872|2021-06-25 13:55:41|2021-06-28 08:48:25|          98|         265|      N|                  B02872|  66.8788888888889|
|              B02765|2021-06-22 12:09:45|2021-06-23 13:42:44|         188|         198|      N|                  B02765|25.549722222222222|
|              B02879|2021-06-27 10:32:29|2021-06-28 06:31:20|          78|         169|      N|                  B02879|19.980833333333333|
|              B02800|2021-06-26 22:37:11|2021-06-27 16:49:01|         263|          36|      N|                    null|18.197222222222223|
|            

In [76]:
zone = spark.read.parquet('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/zones')

In [78]:
zone.registerTempTable('zone')

/home/Дмитрий/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [85]:
zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [91]:
spark.sql(
"""
select zone, count(*) as n
from spark_table st
left join zone z on st.PULocationID = z.LocationID
group by zone
order by n desc
limit 1
"""
).show()

+-------------------+------+
|               zone|     n|
+-------------------+------+
|Crown Heights North|231279|
+-------------------+------+



In [93]:
spark.sql(
'''
show views
'''
).show()

+---------+-----------+-----------+
|namespace|   viewName|isTemporary|
+---------+-----------+-----------+
|         |spark_table|       true|
|         |       zone|       true|
+---------+-----------+-----------+

